Exercise 1

The table contains tree indixes for the table circuits:

    idx_17102_primary (PRIMARY KEY, btree, circuitid)
    idx_17102_url (UNIQE, btree url)
    circuits_position_idx (gist, position)
    
The first one and sedond one is a B tree type.
B trees can handle equality and rage queries which is smart if you need to compare, seach and find fast.
B trees indexes can also be used to retrieve data in sorted order.

The amout of space taken from the two indices is 16 kB.

The last index is a GiST probably because it is a position and a position consists of two coordinates. 
GiST can be usted to find two-dimensional geometric data types.

The amout of space taken by the index is 8192 bytes.
    

In [29]:
%sql SELECT races.date, surname, dob, milliseconds, races.name racename, circuits.name FROM results JOIN drivers USING(driverid) JOIN races USING (raceid) JOIN circuits USING (circuitid) WHERE dob >= '1980-01-01' and rank = 1 LIMIT 10;

10 rows affected.


date,surname,dob,milliseconds,racename,name
2008-03-16,Kovalainen,1981-10-19,5708630,Australian Grand Prix,Albert Park Grand Prix Circuit
2008-04-06,Kovalainen,1981-10-19,5493759,Bahrain Grand Prix,Bahrain International Circuit
2008-08-24,Massa,1981-04-25,5732339,European Grand Prix,Valencia Street Circuit
2008-10-12,Massa,1981-04-25,5468050,Japanese Grand Prix,Fuji Speedway
2008-10-19,Hamilton,1985-01-07,5517403,Chinese Grand Prix,Shanghai International Circuit
2008-11-02,Massa,1981-04-25,5651435,Brazilian Grand Prix,Autódromo José Carlos Pace
2007-04-08,Hamilton,1985-01-07,5552487,Malaysian Grand Prix,Sepang International Circuit
2007-04-15,Massa,1981-04-25,5607515,Bahrain Grand Prix,Bahrain International Circuit
2007-05-13,Massa,1981-04-25,5496230,Spanish Grand Prix,Circuit de Barcelona-Catalunya
2007-05-27,Alonso,1981-07-29,6029329,Monaco Grand Prix,Circuit de Monaco


In [35]:
%sql EXPLAIN ANALYZE SELECT races.date, surname, dob, milliseconds, races.name racename, circuits.name FROM results JOIN drivers USING(driverid) JOIN races USING (raceid) JOIN circuits USING (circuitid) WHERE dob >= '1980-01-01' and rank = 1 LIMIT 10;

19 rows affected.


QUERY PLAN
Limit (cost=25.68..429.39 rows=10 width=62) (actual time=0.150..0.236 rows=10 loops=1)
-> Nested Loop (cost=25.68..752.35 rows=18 width=62) (actual time=0.149..0.235 rows=10 loops=1)
-> Nested Loop (cost=25.54..749.18 rows=18 width=50) (actual time=0.147..0.223 rows=10 loops=1)
-> Hash Join (cost=25.26..737.47 rows=18 width=27) (actual time=0.141..0.205 rows=10 loops=1)
Hash Cond: (results.driverid = drivers.driverid)
-> Seq Scan on results (cost=0.00..708.96 rows=257 width=24) (actual time=0.007..0.068 rows=23 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 438
-> Hash (cost=24.51..24.51 rows=60 width=19) (actual time=0.128..0.128 rows=61 loops=1)
Buckets: 1024 Batches: 1 Memory Usage: 12kB


It makes 9 calls and it takes 4.363 ms time to perform the query.

In [31]:
%sql CREATE VIEW race_winners AS SELECT races.date, surname, dob, milliseconds, races.name racename, circuits.name FROM results JOIN drivers USING(driverid) JOIN races USING (raceid) JOIN circuits USING (circuitid) WHERE dob >= '1980-01-01' and rank = 1 LIMIT 10;

Done.


[]

In [33]:
%sql CREATE MATERIALIZED VIEW race_winners_cache AS SELECT * FROM race_winners;

10 rows affected.


[]

In [34]:
%sql EXPLAIN ANALYSE SELECT * FROM race_winners_cache;

3 rows affected.


QUERY PLAN
Seq Scan on race_winners_cache (cost=0.00..10.50 rows=50 width=1564) (actual time=0.004..0.005 rows=10 loops=1)
Planning time: 0.070 ms
Execution time: 0.015 ms


The execution time has changes a lot and is much faster than before because the statement are calling the cache instead of executing the query again. It is just a snapshot of a former executed statement.